# **Data Collectors and Landing Zone**

### *Required packages:*

In [1]:
# !pip install opendatasets
# !pip install kaggle
# !pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.6/84.6 KB 831.5 kB/s eta 0:00:00 kB/s eta 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 KB 2.0 MB/s eta 0:00:00 MB/s eta 0:00:01
  Using cached click-8.1.7-py3-none-any.whl (97 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.8/162.8 KB 1.9 MB/s eta 0:00:00 MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 KB 2.2 MB/s eta 0:00:00 MB/s eta 0:00:01
  Using cached requests-2.31.0-py3-none-any.whl (62 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 KB 1.7 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 KB 1.9 MB/s eta 0:00:00 MB/s eta 0:00:01
  Using cached charset_normalizer-3.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (142 kB)
  Using cached idna-3.6-py3-none-any.whl (61 kB)
Using legacy 'setup.py install' for kaggle, since package 'wheel' is not installed.
  Running setup.py in

In [2]:
import numpy as np
import pandas as pd
import requests
import os
import opendatasets as od
import csv
import openpyxl

### Kaggle Configuration - Drive

(Requereix del fitxer `kaggle_fitxer.json`)

In [7]:
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/home/oscarmolina/.kaggle’: File exists


In [8]:
!cp ./kaggle_fitxer.json ~/.kaggle/kaggle.json

cp: cannot stat '/kaggle_fitxer.json': No such file or directory


In [ ]:
!cd /kaggle && ls

input  kaggle.json


### CODE

#### Downloading datasets

##### UNECE

In [5]:
# URL del dataset UNECE
unece_db = pd.read_json('https://query.data.world/s/qd6kjqguuvb573mffbpfysgpudi2pv?dws=00000')
unece_db.to_csv('unece_db.csv', index=False)
!mv ./unece_db.csv ../data/unece_db.csv

##### Kaggle - Mental Health

In [6]:
# API Kaggle Mental Health
!kaggle datasets download  -d 'thedevastator/uncover-global-trends-in-mental-health-disorder'

Traceback (most recent call last):
  File "/home/oscarmolina/.venv/bda/bin/kaggle", line 33, in <module>
    sys.exit(load_entry_point('kaggle==1.6.6', 'console_scripts', 'kaggle')())
  File "/home/oscarmolina/.venv/bda/bin/kaggle", line 25, in importlib_load_entry_point
    return next(matches).load()
  File "/usr/lib/python3.10/importlib/metadata/__init__.py", line 171, in load
    module = import_module(match.group('module'))
  File "/usr/lib/python3.10/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1050, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 992, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "<frozen importlib._bootstrap>", line 1050, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_loa

In [ ]:
#!sudo apt-get install unzip
!unzip uncover-global-trends-in-mental-health-disorder.zip -d data/

##### USA Census

In [9]:
# API United States Census
variables = "NAME","YR","AREA_KM2","CBR","CDR","DEATHS","E0","GRR","IMR","MEDAGE","MR0_4","POP_DENS","genc standard countries and areas"
url_base = "https://api.census.gov/data/timeseries/idb/5year?get=NAME,YR,AREA_KM2,CBR,CDR,DEATHS,E0,GRR,IMR,MEDAGE,MR0_4,POP_DENS&for=genc%20standard%20countries%20and%20areas:*".format(variables)

# Llista per guardar les dades
data = []

# Fem una sol·licitud a l'API per agafar dades des de l'any 1990 fins al 2019
for year in range(1990, 2019):
    url = "{}&time={}".format(url_base, year)
    response = requests.get(url)
    if response.status_code == 200:
        data.extend(response.json()[1:])

# Guardar los datos en un archivo CSV
csv_filename = "dades_demogr_census.csv"
with open(csv_filename, "w", newline="") as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["NAME","YR","AREA_KM2","CBR","CDR","DEATHS","E0","GRR","IMR","MEDAGE","MR0_4","POP_DENS","genc standard countries and areas", "code"])
    csv_writer.writerows(data)

print("Les dades s'han guardat a", csv_filename)

!mv ./dades_demogr_census.csv ../data/dades_demogr_census.csv


KeyboardInterrupt: 

#### Storing files in a suitable format (Parquet)


In [58]:
# Unece_db
unece_db.to_parquet('/content/data/unece_db.parquet')

# Kaggle db
kaggle_db = pd.read_csv('/content/data/Mental health Depression disorder Data.csv', decimal=',')
kaggle_db.to_parquet('/content/data/kaggle_db.parquet')

# United State Census db
us_census_db = pd.read_csv('/content/data/dades_demogr_census.csv')
us_census_db.to_parquet('/content/data/us_census_db.parquet')

<ipython-input-58-a336a83c75fc>:5: DtypeWarning: Columns (7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  kaggle_db = pd.read_csv('/content/data/Mental health Depression disorder Data.csv', decimal=',')
